# 5 minute tutorial

The easiest way to use `pydap` is to use it to access remote data hosted on [OPeNDAP](https://www.opendap.org/) servers. You can use `pydap`'s `open_url` directly, or better use `pydap` as an engine for `xarray`. Both ways are equivalent, but xarray allows for [OPeNDAP](https://www.opendap.org/) users to exploit many of [Pangeo](https://pangeo.io/)'s modern capabilities for scalable computing.


## OPeNDAP - the vision
The original vision of [OPeNDAP](https://www.opendap.org/) ([Cornillion, et al 1993](https://zenodo.org/records/10610992)) was to make the equivalency:

$ \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \boxed{\text{URL} \approx \text{Remote Dataset} }$


Furthermore, 

$ \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \boxed{\text{URL + Constraints} \approx \text{Subset of Remote Dataset}} $


Here, we demonstrate this. For this short tutorial we will access a remote dataset hosted on [OPeNDAP's Hyrax server](https://www.opendap.org/software/hyrax-data-server/). For more information about [OPeNDAP](https://www.opendap.org/) and Hyrax you can go to the official [OPeNDAP documentation](https://opendap.github.io/documentation/UserGuideComprehensive.html).

The remote dataset that will be used in this tutorial can be inspected via the browser [HERE](http://test.opendap.org:8080/opendap/tutorials/20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc.dmr.html)


In [1]:
from pydap.client import open_url
import xarray as xr
import numpy as np

We define a URL pointing to a remote dataset.

In [2]:
url = "http://test.opendap.org:8080/opendap/tutorials/20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc"

### pydap approach
We access the remote dataset via pydap

In [3]:
pydap_ds = open_url(url, protocol='dap4')

Note the extra argument `protocol='dap4'`. One could also pass `protocol='dap2'`. We can inspect the contents of the dataset as follows:

In [4]:
pydap_ds.tree()

.20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
├──time
├──lat
├──lon
├──analysed_sst
├──analysis_error
├──mask
├──sea_ice_fraction
├──dt_1km_data
└──sst_anomaly


In [5]:
pydap_ds.attributes

{'Conventions': 'CF-1.7',
 'title': 'Daily MUR SST, Final product',
 'summary': 'A merged, multi-sensor L4 Foundation SST analysis product from JPL.',
 'references': 'http://podaac.jpl.nasa.gov/Multi-scale_Ultra-high_Resolution_MUR-SST',
 'institution': 'Jet Propulsion Laboratory',
 'history': 'created at nominal 4-day latency; replaced nrt (1-day latency) version.',
 'comment': 'MUR = "Multi-scale Ultra-high Resolution"',
 'license': 'These data are available free of charge under data policy of JPL PO.DAAC.',
 'id': 'MUR-JPL-L4-GLOB-v04.1',
 'naming_authority': 'org.ghrsst',
 'product_version': '04.1',
 'uuid': '27665bc0-d5fc-11e1-9b23-0800200c9a66',
 'gds_version_id': '2.0',
 'netcdf_version_id': '4.1',
 'date_created': '20220609T072338Z',
 'start_time': '20220531T090000Z',
 'stop_time': '20220531T090000Z',
 'time_coverage_start': '20220530T210000Z',
 'time_coverage_end': '20220531T210000Z',
 'file_quality_level': 3,
 'source': 'MODIS_T-JPL, MODIS_A-JPL, AMSR2-REMSS, AVHRRMTB_G-NAVO,

### **NOTE**: 

No data has been downloaded yet. `pydap` reads the metadata of the URL to create the pydap dataset. 

We can further inspect some of the variables attributes.

In [6]:
pydap_ds['sst_anomaly'].shape

(1, 17999, 36000)

In [7]:
print('This array occupies: ', pydap_ds['sst_anomaly'].nbytes/1e9, '[GBs] in memory')

This array occupies:  1.295928 [GBs] in memory


Each variable contains `CF-compliant metadata` that can be recognized by various APIs, such as `scale_factor`, `offsets` and `_FillValue`. These parameters are necessary to mask over land areas, and scale values. Some APIs like xarray can recognize these, while for others a user must manually transform the data.



In [8]:
pydap_ds['sst_anomaly'].attributes

{'long_name': 'SST anomaly from a seasonal SST climatology based on the MUR data over 2003-2014 period',
 'units': 'kelvin',
 '_FillValue': -32768,
 'add_offset': 0.0,
 'scale_factor': 0.001,
 'valid_min': -32767,
 'valid_max': 32767,
 'comment': 'anomaly reference to the day-of-year average between 2003 and 2014',
 'coordinates': 'lon lat'}

You can read more about `NetCDF Climate and Forcasts (CF) Metadata Conventions` [HERE](https://cfconventions.org/cf-conventions/cf-conventions.html).


### **Downloading the Array**


You can trigger a download on-the-fly as needed. **However** in almost all cases `only a subset of an entire dataset is needed`. You can download only the piece you want, by slicing the array as follows:

In [9]:
%%time
array = pydap_ds['sst_anomaly'][0, 0:10, 0:10]

CPU times: user 8.39 ms, sys: 2.74 ms, total: 11.1 ms
Wall time: 1.55 s


In [10]:
np.shape(array)

(1, 10, 10)

With the above command, all the data-array has been downloaded into memory and assigned to the variable `array`. However, the variable `array` is not a numpy array, but rather a `BaseType` of `pydap`'s model:

In [11]:
type(array)

pydap.model.BaseType

To extract the numpy array from `pydap`'s `BaseType` do:

In [12]:
data = array.data

In [13]:
type(data)

numpy.ndarray

### Using server-side processing

Because data is hosted on Hyrax, you can exploit server-side processing that occurs local to the data to perform subsetting. [OPeNDAP](https://www.opendap.org/) servers support subsetting by adding `Constraint Expressions` to the `URL`.

In this scenario were we want a subset of the variable `sst_anomaly`, we can request it directly to [OPeNDAP](https://www.opendap.org/)'s Hyrax server using the following syntax:



```python
<OPeNDAP_URL> + "?dap4.ce=\sst_anomaly[0][0:1:9][0:1:9]"
```


In [14]:
CE = "?dap4.ce=/sst_anomaly[0][0:1:9][0:1:9]"

In [15]:
pydap_ds = open_url(url+CE, protocol='dap4')

In [16]:
pydap_ds.tree()

.20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
└──sst_anomaly


In [17]:
pydap_ds['sst_anomaly'].shape

(1, 10, 10)

### xarray approach

`pydap`'s `open_url` can be used internally within `xarray`, by defining as an extra parameter when creating an `xarray` Dataset. The extra parameter is:

```python
engine='pydap'
```

Moreoever, we can combine the `server-side` processing that occurs **local to the data on the OPeNDAP server**, with `xarray`.

### DAP2 vs DAP4

There are some differences between the `DAP2` and `DAP4` [OPeNDAP](https://www.opendap.org/) model that go beyond this 5 minute intro. We will simply restrict to say that `DAP4` is newer and that will be the focus of this short tutorial. `pydap` accepts a `protocol` argument which specifies `"dap2"` vs `"dap4"`. `xarray` however does not. 

We can specify `DAP4` as the protocol that the [OPeNDAP](https://www.opendap.org/) server will implement when requesting access and server-side processing to the data. We do this by passing a URL that begins with `dap4`. For example in this case the following URL can be passed within `xarray` and `pydap` will recognize the `DAP4` protocol specification:



In [18]:
'dap4'+url[4:]

'dap4://test.opendap.org:8080/opendap/tutorials/20220531090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'

In [19]:
dataset = xr.open_dataset('dap4'+url[4:], engine='pydap')
dataset

<xarray.Dataset> Size: 29GB
Dimensions:           (time: 1, lat: 17999, lon: 36000)
Coordinates:
  * time              (time) datetime64[ns] 8B 2022-05-31T09:00:00
  * lat               (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.98 89.99
  * lon               (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0
Data variables:
    analysed_sst      (time, lat, lon) float64 5GB ...
    analysis_error    (time, lat, lon) float64 5GB ...
    mask              (time, lat, lon) float32 3GB ...
    sea_ice_fraction  (time, lat, lon) float64 5GB ...
    dt_1km_data       (time, lat, lon) timedelta64[ns] 5GB ...
    sst_anomaly       (time, lat, lon) float64 5GB ...
Attributes: (12/47)
    Conventions:                CF-1.7
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    ...                         ...
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid

Lastly, we can also pass a URL with a `Constraint Expression` directly onto `xarray` directly as follows:

In [20]:
CE = "?dap4.ce=/time;/lat;/lon;/sst_anomaly"

In [21]:
dataset_ce = xr.open_dataset(url+CE, engine='pydap')
dataset_ce

<xarray.Dataset> Size: 5GB
Dimensions:      (time: 1, lat: 17999, lon: 36000)
Coordinates:
  * time         (time) datetime64[ns] 8B 2022-05-31T09:00:00
  * lat          (lat) float32 72kB -89.99 -89.98 -89.97 ... 89.97 89.98 89.99
  * lon          (lon) float32 144kB -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
Data variables:
    sst_anomaly  (time, lat, lon) float64 5GB ...
Attributes: (12/47)
    Conventions:                CF-1.7
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    ...                         ...
    project:                    NASA Making Earth Science Data Records for Us...
    publisher_name:             GHRSST Project Office
    publisher_url:              http://www.ghrsst.org
    publisher_email:            ghrsst-po@nceo.ac.uk
    processing_level:           L4
    cdm_data_type:              grid